In [1]:
from PubMed.PmcScraper import PmcScraper
import os

# need .env file to store ncbi api key
pmc_scraper = PmcScraper(os.environ['NCBI_API_KEY'])

# search based on keyword and retrieve the desired amount of articles
search_results = pmc_scraper.search_pmc(search_term="gut microbiota", number_of_articles=100)

# extract the full text version of articles using pmcids
full_text_articles = pmc_scraper.fetch_full_text(search_results['esearchresult']['idlist'])

# extract the desired section of articles
method_sections = pmc_scraper.extract_section(full_text_articles, 'method')

# print as dictionary
method_dict = dict(zip(search_results['esearchresult']['idlist'], method_sections))

In [2]:
method_dict

{'11193528': 'MATERIALS AND METHODS The detailed protocols are available in Supplemental Materials (11–15); all Supplemental material is available at https://doi.org/10.6084/m9.figshare.24556174.',
 '11193507': 'Materials and Methods:NAFLD was induced in male Wistar rats (aged 6–8 weeks) by feeding them a high-fat diet (HFD) for 6 weeks. Subsequently, the rats were divided into four groups, with Group 1 continuing on HFD, while groups 2, 3, and 4 received HFD supplemented with saroglitazar, curcumin, and both saroglitazar and curcumin, respectively. We evaluated the expression of Nrf2, ERK1/2, NOX1,2,4, antioxidant enzymes, PPARα, γ, and genes regulating lipid metabolism in the liver. Histopathology of liver tissue was also examined. Furthermore, we analyzed serum levels of lipid profiles and hepatic enzymes.',
 '11193499': 'Materials and Methods To induce NAFLD, we prepared a high-fat emulsion following the methodology outlined in the studies conducted by Zou et al. (10). This emulsio

In [ ]:

# nlp = spacy.load("en_core_web_lg")

# doc = nlp("Mr Tan Ah Kow has a history of medical conditions. He has had hypertension and hyperlipidemia since 1990 and suffered several strokes in 2005. He subsequentlydeveloped heart problems (cardiomyopathy), cardiac failure and chronic renal diseaseand was treated in ABC Hospital.He was last admitted to the ABC Hospital on 1 April 2010 till 15 April 2010, duringwhich he was diagnosed to have suffered from a stroke. This was confirmed by CTand MRI brain scans.Thereafter, he was transferred to XYZ Hospital for stroke rehabilitation on 15 April2010.After that, Mr Tan was referred to Blackacre Hospital for follow-up treatment from inNovember 2010. The clinical impression was that he was manifesting behavioural andpsychological symptoms secondary to Dementia")

# for ent in doc.ents:
#     print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [3]:
import spacy

# products are generally followed by company names
nlp = spacy.load("en_core_web_trf")

entity_dict = {}
for key, value in method_dict.items():
    doc = nlp(value)
    entities = []
    for ent in doc.ents:
        if ent.label_ == 'PRODUCT':
            entities.append(ent.text)
    entity_dict[key] = entities

In [4]:

doc = nlp(method_dict['11192171'])

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

≥98% 97 101 PERCENT
933.13 Da 126 135 QUANTITY
Chengdu Purifa Technology Development Co. Ltd 156 201 ORG
Chengdu 203 210 GPE
China 212 217 GPE
0216011010 254 264 CARDINAL
36 kDa–50 kDa 270 283 QUANTITY
MP Biomedicals 304 318 ORG
Shanghai 320 328 GPE
China 330 335 GPE
398.39 Da 410 419 QUANTITY
FD4 439 442 PRODUCT
Sigma-Aldrich 482 495 ORG
Darmstadt 497 506 GPE
Germany 508 515 GPE
T6113 527 532 PRODUCT
98% 555 558 PERCENT
548.64 Da 584 593 QUANTITY
TOPSCIENCE 613 623 ORG
Shanghai 625 633 GPE
China 635 640 GPE
693520 663 669 PRODUCT
MilliporeSigma 707 721 ORG
Burlington 723 733 GPE
MA 735 737 GPE
USA 739 742 GPE
the American Type Culture Collection 856 892 ORG
ATCC 894 898 ORG
Manassas 900 908 GPE
VA 910 912 GPE
USA 914 917 GPE
CT26 931 935 PRODUCT
Roswell Park Memorial Institute 959 990 ORG
11875085 1019 1027 PRODUCT
Gibco 1029 1034 GPE
NY 1036 1038 GPE
USA 1040 1043 GPE
10% 1063 1066 PERCENT
10099158 1087 1095 PRODUCT
Gibco 1097 1102 GPE
NY 1104 1106 GPE
USA 1108 1111 GPE
5% 1181 1183 

In [5]:
# extract the full text version of articles using pmcids
full_text_articles = pmc_scraper.fetch_full_text(['11189522'])

# extract the desired section of articles
method_sections = pmc_scraper.extract_section(full_text_articles, 'method')

In [6]:
method_sections

['Materials and methods Materials All chemicals were obtained from Sigma (St Louis, MO, USA) unless otherwise stated. 2’-fucosyllactose (2’FL) and 3-fucosyllactose (3FL) were obtained from Glycom/DSM (Esbjerg, Denmark). Blood group A type II (BgA), Blood group B type II (BgB), Blood group H type II (BgH) and LewisY (LeY) were obtained from Elicityl (Crolles, France). Lewis A trisaccharide (LeA), 3′-sialyl Lewis A (sLeA), Lewis X trisaccharide (LeX), 3’-sialyl Lewis X (sLeX), 2-acetamido-2-deoxy-6-O-(α-l-fucopyranosyl)-d-glucopyranose (6FN), 2-acetamido-2-deoxy-4-O-(α-l-fucopyranosyl)-d-glucopyranose (4FN), 2-acetamido-2-deoxy-3-O-(α-l-fucopyranosyl)-d-glucopyranose (3FN), 4-nitrophenyl α-l-fucopyranoside (pNP-Fuc), 2-Chloro-4-nitrophenyl-αl-fucopyranoside (CNP-Fuc), 2-Chloro-4-nitrophenol (CNP) and N-acetyllactosamine (LacNAc) were obtained from Biosynth Ltd (Compton, UK). FA2G2 N-glycan was from Ludger (Oxford, UK). IgG was purified from human serum using the protein A IgG purificatio

In [7]:
doc = nlp(method_sections[0])
for ent in doc.ents:
    if ent.label_ == 'ORG' or ent.label_ == 'PRODUCT':
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

Sigma 65 70 ORG
Glycom/DSM 187 197 ORG
Elicityl 340 348 ORG
Biosynth Ltd 856 868 ORG
FA2G2 N-glycan 884 898 PRODUCT
Ludger 908 914 ORG
Thermofisher 1009 1021 ORG
Newcastle University 1178 1198 ORG
Sigma 1281 1286 ORG
R. gnavus E1 1348 1360 PRODUCT
R. gnavus 1383 1392 PRODUCT
CAZy 1586 1590 ORG
GH29 1633 1637 PRODUCT
CAZy 1657 1661 ORG
the Enzyme Function Initiative-Enzyme Similarity Tool 1861 1914 PRODUCT
Cytoscape 3.9.1 2024 2039 PRODUCT
GH29 2045 2049 PRODUCT
GH29BERT 2158 2166 PRODUCT
GH29 2608 2612 PRODUCT
InterPro 2684 2692 ORG
Transformer 2875 2886 PRODUCT
NVIDIA 3844 3850 ORG
A100 40GB 3851 3860 PRODUCT
ProtT5 model29 5763 5777 PRODUCT
GH29BERT 5882 5890 PRODUCT
GH29 6428 6432 PRODUCT
Uniform Manifold Approximation and Projection 6596 6641 PRODUCT
Python 6946 6952 PRODUCT
Prozomix 7311 7319 ORG
NZYTech 7401 7408 ORG
Cytiva 7901 7907 ORG
NuPAGE Novex 8026 8038 PRODUCT
Bis-Tris 8045 8053 PRODUCT
Life Technologies 8060 8077 ORG
NanoDrop 8135 8143 PRODUCT
Thermo Scientific 8145 8162

In [8]:
doc = nlp(method_sections[0])
products = []
for ent in doc.ents:
    if ent.label_ == 'PRODUCT' and ent.text not in products:
        products.append(ent.text)

In [9]:
products

['FA2G2 N-glycan',
 'R. gnavus E1',
 'R. gnavus',
 'GH29',
 'the Enzyme Function Initiative-Enzyme Similarity Tool',
 'Cytoscape 3.9.1',
 'GH29BERT',
 'Transformer',
 'A100 40GB',
 'ProtT5 model29',
 'Uniform Manifold Approximation and Projection',
 'Python',
 'NuPAGE Novex',
 'Bis-Tris',
 'NanoDrop',
 'FLUOstar Omega',
 'HPAEC-PAD',
 'ICS 5000',
 'CarboPac PA1',
 'Savant SpeedVac',
 'a LudgerClean Procainamide Plate',
 'LC-PROC-96',
 'Ultimate 3000',
 'Amazon Speed ETD',
 'an Expression Compact Mass Spectrometer',
 'Plate Express',
 'Advion Mass Express',
 'Bruker Avance',
 'Fucose',
 '6FN',
 '3FN',
 '4FN',
 'Bruker Avance III',
 'FA2G2',
 'xia279',
 'dials80',
 '1ODU',
 'Chainsaw81',
 'ArpWarp82',
 'coot83',
 'refmac84',
 'PDBredo85']

In [10]:
instruct_string = """You are an expert Physical Commercial Product Recognition system.
Your task is to accept a list of strings as input and decide which of these are physical commercial products.
Here is an example of the output format for a list of strings.

List of strings: ['Python', '3M Littmann Stethoscope', 'Omron Blood Pressure Monitor', 'Microsoft Office', 'Braun ThermoScan', 'SPSS']
Answer: ['3M Littmann Stethoscope', 'Omron Blood Pressure Monitor', 'Braun ThermoScan']

Only use this output format. Do not output anything besides this output format.
Output physical commercial products in the order they occur in the input list of strings.

Q: Given the list of strings below, identify the physical commercial products.

List of strings: {}
Answer:
"""

In [11]:
print(instruct_string.format(products))

You are an expert Physical Commercial Product Recognition system.
Your task is to accept a list of strings as input and decide which of these are physical commercial products.
Here is an example of the output format for a list of strings.

List of strings: ['Python', '3M Littmann Stethoscope', 'Omron Blood Pressure Monitor', 'Microsoft Office', 'Braun ThermoScan', 'SPSS']
Answer: ['3M Littmann Stethoscope', 'Omron Blood Pressure Monitor', 'Braun ThermoScan']

Only use this output format. Do not output anything besides this output format.
Output physical commercial products in the order they occur in the input list of strings.

Q: Given the list of strings below, identify the physical commercial products.

List of strings: ['FA2G2 N-glycan', 'R. gnavus E1', 'R. gnavus', 'GH29', 'the Enzyme Function Initiative-Enzyme Similarity Tool', 'Cytoscape 3.9.1', 'GH29BERT', 'Transformer', 'A100 40GB', 'ProtT5 model29', 'Uniform Manifold Approximation and Projection', 'Python', 'NuPAGE Novex', 'Bi

In [12]:
import ollama

stream = ollama.chat(
    model='llama3',
    messages=[{'role': 'user', 'content': instruct_string.format(products)}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

['NuPAGE Novex', 'Bis-Tris', 'NanoDrop', 'FLUOstar Omega', 'HPAEC-PAD', 'ICS 5000', 'CarboPac PA1', 'Savant SpeedVac', 'Ultimate 3000', 'Bruker Avance III', 'Chainsaw81']

In [13]:
stream = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': instruct_string.format(products)}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

 ['NuPAGE Novex', 'NanoDrop', 'FLUOstar Omega', 'ICS 5000', 'CarboPac PA1', 'Savant SpeedVac', 'a LudgerClean Procainamide Plate', 'LC-PROC-96', 'Ultimate 3000', 'Amazon Speed ETD', 'an Expression Compact Mass Spectrometer', 'Plate Express', 'Advion Mass Express', 'Bruker Avance', 'Fucose']

In [14]:
import openai 
openai.api_key = os.environ['OPENAI_API_KEY']

In [15]:
system_prompt = """You are an expert Physical Commercial Product Recognition system.
Your task is to accept a list of strings as input and decide which of these are physical commercial products.
Here is an example of the output format for a list of strings.

List of strings: ['Python', '3M Littmann Stethoscope', 'Omron Blood Pressure Monitor', 'Microsoft Office', 'Braun ThermoScan', 'SPSS']
Answer: ['3M Littmann Stethoscope', 'Omron Blood Pressure Monitor', 'Braun ThermoScan']

Only use this output format. Do not output anything besides this output format.
Output physical commercial products in the order they occur in the input list of strings.
"""

user_prompt = """Q: Given the list of strings below, identify the physical commercial products.

List of strings: {}
Answer:
"""

In [16]:
response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt.format(products)}
        ]
    )

In [17]:
print(response.choices[0].message.content)

['A100 40GB', 'NuPAGE Novex', 'Bis-Tris', 'NanoDrop', 'FLUOstar Omega', 'ICS 5000', 'CarboPac PA1', 'Savant SpeedVac', 'Ultimate 3000', 'Amazon Speed ETD', 'an Expression Compact Mass Spectrometer', 'Plate Express', 'Advion Mass Express', 'Bruker Avance', 'Bruker Avance III']
